In [53]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
#from api_keys import g_key
from api_keys import weather_api_key


#configure gmaps
gmaps.configure(api_key=g_key)

In [54]:
df= pd.read_csv("../WeatherPy/WeatherPy_removed_DidNotFind_Rows.csv")
new_df=pd.DataFrame(df)
new_df.head()
kelvin = new_df["Max Temp"]
new_df["Max Temp"] = 1.8 * (kelvin - 273) + 32

mph =2.23694
new_df["Wind Speed"] =new_df["Wind Speed"]*mph
new_df.head()



,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saint-dizier,48.42,5.08,50.270,71,63,5.816044,FR,1602699436
1,1,albany,42.60,-73.97,68.270,34,0,8.925391,US,1602699190
2,2,eldorado,-23.79,-54.28,99.770,25,31,6.956883,BR,1602699438
3,3,aurillac,44.92,2.45,45.266,93,90,4.697574,FR,1602699440
4,4,biak,-0.91,122.88,78.368,79,99,5.435764,ID,1602699441


fig=gmaps.figure()
heatmap_layer=gmaps.heatmap_layer(
    new_df(['Lat', 'Lng']), weights=new_df['Humidity'], max_intensity=30, point_radius=3.0)
fig.add_layer(heatmap_layer)
fig


In [55]:
fig=gmaps.figure() 
heatmap_layer=gmaps.heatmap_layer( 
    new_df[['Lat', 'Lng']], weights=new_df['Humidity'], 
     point_radius=5.0) 
fig.add_layer(heatmap_layer) 
fig

Figure(layout=FigureLayout(height='420px'))

In [36]:
new_df.dtypes

#new_df_hotels= new_df.loc[(new_df["Max Temp"]>70) & (new_df["Max Temp"]<80) & (new_df["Wind Speed"]<5) & (new_df["Cloudiness"]==0)]
new_df_hotels= new_df.loc[(new_df["Max Temp"]>70) & (new_df["Max Temp"]<73) & (new_df["Country"]=='US')]
len(new_df_hotels)

5

In [37]:
new_df_hotels

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
234,259,fallon,39.47,-118.78,72.266,35,40,6.934514,US,1602699694
255,283,freeport,40.66,-73.58,70.268,40,1,11.408394,US,1602699760
308,339,saint george,37.10,-113.58,70.070,15,1,3.780429,US,1602699866
404,439,pacific grove,36.62,-121.92,72.266,53,1,4.697574,US,1602699990
424,460,brigantine,39.41,-74.36,70.070,43,1,8.052984,US,1602700016


In [65]:

Hotel_df = pd.DataFrame(new_df_hotels["City"])

Hotel_df["Hotel Lat"] =''
Hotel_df["Hotel Lng"] =''
Hotel_df["Hotel Name"] =''
Hotel_df["Hotel Country"] =new_df_hotels['Country']


#Hotel_df.drop('index',axis=1)
Hotel_df1 = pd.DataFrame(Hotel_df,columns=["City","Hotel Lat","Hotel Lng","Hotel Name","Hotel Country"])

Hotel_df1

,City,Hotel Lat,Hotel Lng,Hotel Name,Hotel Country
234,fallon,,,,US
255,freeport,,,,US
308,saint george,,,,US
404,pacific grove,,,,US
424,brigantine,,,,US


In [66]:
Hotel_df1.reset_index(drop=True)

,City,Hotel Lat,Hotel Lng,Hotel Name,Hotel Country
0,fallon,,,,US
1,freeport,,,,US
2,saint george,,,,US
3,pacific grove,,,,US
4,brigantine,,,,US


In [ ]:
#target_type = "bicycle_store"
#seattle_coords = "47.6062095,-122.3320708"
#radius = 8000

# rewrite params dict

#params = {
    #"location": seattle_coords,
    #"types": target_type,
    #"radius": radius,
    #"key": gkey
#}

# Build URL using the Google Maps API
#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#print("Drill #3: A Bike Store in Seattle, WA")

# Run request
#response = requests.get(base_url, params)

# print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
# print(response.url)

#seattle_bikes = response.json()

# Print the JSON (pretty printed)
# print(json.dumps(seattle_bikes, indent=4, sort_keys=True))

# Print the name and address of the first bike shop to appear
#print(seattle_bikes["results"][0]["name"])
#print(seattle_bikes["results"][0]["vicinity"])





In [67]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#base_url_geo = "https://maps.googleapis.com/maps/api/geocode/json"









#api.openweathermap.org/data/2.5/weather?q=London&appid={API key}
#city_info = requests.get(url +city + '&appid='+weather_api_key ).json()



#1. google places to get hotel name,  and hotel coridinates (lat, lng). 
#2. use lat and long to use openweather to get city and country 

# geocoordinates

target_radius = 8000
target_type = "lodging"

# set up a parameters dictionary
  
lat=[]
lng=[]
    
for index, row in new_df_hotels.iterrows():
    
    #time.sleep(1.1)
    #city = row['City']
    lat = row['Lat']
    lng = row['Lng']   
    coords=f"{lat}, {lng}"
    #print(coords)
    
    params = {"location":  coords ,"radius": target_radius, "type": target_type, "key": g_key}    
    response_place = requests.get(base_url, params=params).json()
    #Hotel_df1.loc[index, "Hotel Name"] =["results"][0]["name"]
    Hotel_df1.loc[index, "Hotel Name"] =response_place["results"][0]["name"]
    Hotel_df1.loc[index, "Hotel Lat"] = response_place["results"][0]["geometry"]["location"]["lat"]
    Hotel_df1.loc[index, "Hotel Lng"] = response_place["results"][0]["geometry"]["location"]["lng"]

    
    
    #Hotel_df1.loc[index, "Hotel Address"] = response_place["results"][0]["plus_code"]["compound_code"]
    
    
    
    
    #params_geo = {"address": city ,"radius": target_radius, "type": target_type, "key": g_key}    
    #response_place = requests.get(base_url, params=params_geo)
    #place = response_place.json()
    #Hotel_df1.loc[index, "Hotel Lat"] = place["results"][0]["geometry"]["location"]["lat"]
    #Hotel_df1.loc[index, "Hotel Lng"] = place["results"][0]["geometry"]["location"]["lng"]
    
    
    #Hotel_df1.loc[index, "Hotel Name"] = place["results"][0]["name"]
    

        
        
#params = {"address": target_city, "key": gkey}    
#response = requests.get(base_url, params=params).json()


# Extract lat/lng
#lat = response["results"][0]["geometry"]["location"]["lat"]
#lng = response["results"][0]["geometry"]["location"]["lng"]    
    
    
    
    
    
#for city in cities:
    #params = {"address": city,"radius": target_radius, "type": target_type, "key": g_key}
    #response = requests.get(base_url, params=params).json()
    #lat = seattle_geo["results"][0]["geometry"]["location"]["lat"]
    #lng = seattle_geo["results"][0]["geometry"]["location"]["lng"]

#code for google_place    
#params = {"location": "39.47, -118.78" ,"radius": target_radius, "type": target_type, "key": g_key}    
#response_place = requests.get(base_url, params=params).json()
#print(json.dumps(response, indent=4, sort_keys=True))
#print(response["results"][0]["name"])
#print(response["results"][0]["vicinity"])
#print(response["results"][0]["plus_code"]["compound_code"])




print(Hotel_df1)
print(new_df_hotels)
print(json.dumps(response_place, indent=4, sort_keys=True))

              City Hotel Lat Hotel Lng                        Hotel Name  \
234         fallon   39.4746   -118.79  Best Western Fallon Inn & Suites   
255       freeport   40.6578  -73.6488     Best Western Mill River Manor   
308   saint george    37.089  -113.584       Best Western Plus Abbey Inn   
404  pacific grove   36.6256  -121.917                  Lovers Point Inn   
424     brigantine   39.3699  -74.4126                   Flagship Resort   

    Hotel Country  
234            US  
255            US  
308            US  
404            US  
424            US  
     Unnamed: 0           City    Lat     Lng  Max Temp  Humidity  Cloudiness  \
234         259         fallon  39.47 -118.78    72.266        35          40   
255         283       freeport  40.66  -73.58    70.268        40           1   
308         339   saint george  37.10 -113.58    70.070        15           1   
404         439  pacific grove  36.62 -121.92    72.266        53           1   
424         460   

NameError: name 'weather_api_key' is not defined

In [68]:
#Hotel Name, City, and Country
Hotel_df1

,City,Hotel Lat,Hotel Lng,Hotel Name,Hotel Country
234,fallon,39.4746,-118.79,Best Western Fallon Inn & Suites,US
255,freeport,40.6578,-73.6488,Best Western Mill River Manor,US
308,saint george,37.089,-113.584,Best Western Plus Abbey Inn,US
404,pacific grove,36.6256,-121.917,Lovers Point Inn,US
424,brigantine,39.3699,-74.4126,Flagship Resort,US


In [81]:
loc=[]

for lat, long in zip(Hotel_df1['Hotel Lat'], Hotel_df1['Hotel Lng']):
    concat = f"{lat} + {long} 
    loc.append(concat)
print(loc)    
    

#symbols = gmaps.symbol_layer(
        #loc, fill_color='red', stroke_color='red')
#fig.add_layer(symbols)

[-79.3154231, -32.991008900000004, -76.4952424, -85.29185220000001, -35.04273680000001]
